In [1]:
!python -V

Python 3.9.13


In [2]:
import pandas as pd
import fastparquet
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, Lasso

In [3]:
'''
# Read the Parquet file
df_train = pd.read_parquet('./data/green_tripdata_2021-01.parquet')
df_val = pd.read_parquet('./data/green_tripdata_2021-05.parquet')
df_test = pd.read_parquet('./data/green_tripdata_2021-07.parquet')
'''

"\n# Read the Parquet file\ndf_train = pd.read_parquet('./data/green_tripdata_2021-01.parquet')\ndf_val = pd.read_parquet('./data/green_tripdata_2021-05.parquet')\ndf_test = pd.read_parquet('./data/green_tripdata_2021-07.parquet')\n"

In [4]:
'''
# Save as CSV
df_train.to_csv('./data/green_tripdata_2021-01.csv', index=False)
df_val.to_csv('./data/green_tripdata_2021-02.csv', index=False)
df_test.to_csv('./data/green_tripdata_2021-07.csv', index=False)
'''

"\n# Save as CSV\ndf_train.to_csv('./data/green_tripdata_2021-01.csv', index=False)\ndf_val.to_csv('./data/green_tripdata_2021-02.csv', index=False)\ndf_test.to_csv('./data/green_tripdata_2021-07.csv', index=False)\n"

In [5]:
df_train = pd.read_csv('./data/green_tripdata_2021-01.csv')
df_val = pd.read_csv('./data/green_tripdata_2021-02.csv')
df_test = pd.read_csv('./data/green_tripdata_2021-07.csv')

C:\Users\Sam Berchie\AppData\Local\Temp\ipykernel_19692\3898608297.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv('./data/green_tripdata_2021-01.csv')
C:\Users\Sam Berchie\AppData\Local\Temp\ipykernel_19692\3898608297.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_val = pd.read_csv('./data/green_tripdata_2021-02.csv')
C:\Users\Sam Berchie\AppData\Local\Temp\ipykernel_19692\3898608297.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_test = pd.read_csv('./data/green_tripdata_2021-07.csv')


In [6]:
df_train.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2021-01-01 00:15:56,2021-01-01 00:19:52,N,1.0,43,151,1.0,1.01,5.5,0.5,0.5,0.00,0.0,NaN,0.3,6.80,2.0,1.0,0.00
1,2,2021-01-01 00:25:59,2021-01-01 00:34:44,N,1.0,166,239,1.0,2.53,10.0,0.5,0.5,2.81,0.0,NaN,0.3,16.86,1.0,1.0,2.75
2,2,2021-01-01 00:45:57,2021-01-01 00:51:55,N,1.0,41,42,1.0,1.12,6.0,0.5,0.5,1.00,0.0,NaN,0.3,8.30,1.0,1.0,0.00
3,2,2020-12-31 23:57:51,2021-01-01 00:04:56,N,1.0,168,75,1.0,1.99,8.0,0.5,0.5,0.00,0.0,NaN,0.3,9.30,2.0,1.0,0.00
4,2,2021-01-01 00:16:36,2021-01-01 00:16:40,N,2.0,265,265,3.0,0.00,-52.0,0.0,-0.5,0.00,0.0,NaN,-0.3,-52.80,3.0,1.0,0.00


In [7]:
df_val.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2,2021-05-01 00:49:47,2021-05-01 00:52:58,N,1.0,65,25,1.0,0.56,4.0,0.5,0.5,1.06,0.0,NaN,0.3,6.36,1.0,1.0,0.0
1,2,2021-05-01 01:08:45,2021-05-01 01:25:36,N,1.0,25,223,1.0,9.37,27.5,0.5,0.5,7.20,0.0,NaN,0.3,36.00,1.0,1.0,0.0
2,2,2021-05-01 00:12:39,2021-05-01 00:17:09,N,1.0,134,135,2.0,1.44,6.0,0.5,0.5,0.00,0.0,NaN,0.3,7.30,2.0,1.0,0.0
3,2,2021-05-01 00:22:24,2021-05-01 00:33:03,N,1.0,82,226,1.0,1.78,9.0,0.5,0.5,3.80,0.0,NaN,0.3,14.10,1.0,1.0,0.0
4,2,2021-05-01 00:53:41,2021-05-01 00:57:00,N,1.0,41,166,1.0,0.50,4.0,0.5,0.5,0.00,0.0,NaN,0.3,5.30,2.0,1.0,0.0


In [8]:
df_test.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,1,2021-07-01 00:30:52,2021-07-01 00:35:36,N,1.0,74,168,1.0,1.20,6.0,0.5,0.5,0.00,0.0,NaN,0.3,7.30,2.0,1.0,0.0
1,2,2021-07-01 00:25:36,2021-07-01 01:01:31,N,1.0,116,265,2.0,13.69,42.0,0.5,0.5,0.00,0.0,NaN,0.3,43.30,2.0,1.0,0.0
2,2,2021-07-01 00:05:58,2021-07-01 00:12:00,N,1.0,97,33,1.0,0.95,6.5,0.5,0.5,2.34,0.0,NaN,0.3,10.14,1.0,1.0,0.0
3,2,2021-07-01 00:41:40,2021-07-01 00:47:23,N,1.0,74,42,1.0,1.24,6.5,0.5,0.5,0.00,0.0,NaN,0.3,7.80,2.0,1.0,0.0
4,2,2021-07-01 00:51:32,2021-07-01 00:58:46,N,1.0,42,244,1.0,1.10,7.0,0.5,0.5,0.00,0.0,NaN,0.3,8.30,2.0,1.0,0.0


In [10]:
len(df_train), len(df_val)

(76518, 88180)

In [9]:
def read_dataframe(filename):
    df = pd.read_csv(filename)
    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime) 
    df.lpep_pickup_datetime  = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 0) & (df.duration <= 60)] 

    categorical = ['PULocationID', 'DOLocationID']
    df.loc[:, categorical] = df[categorical].astype(str)
    
    return df

In [27]:
df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [29]:
df

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,duration
0,2,2021-01-01 00:15:56,2021-01-01 00:19:52,N,1.0,43,151,1.0,1.01,5.50,...,0.5,0.00,0.00,NaN,0.3,6.80,2.0,1.0,0.00,3.933333
1,2,2021-01-01 00:25:59,2021-01-01 00:34:44,N,1.0,166,239,1.0,2.53,10.00,...,0.5,2.81,0.00,NaN,0.3,16.86,1.0,1.0,2.75,8.750000
2,2,2021-01-01 00:45:57,2021-01-01 00:51:55,N,1.0,41,42,1.0,1.12,6.00,...,0.5,1.00,0.00,NaN,0.3,8.30,1.0,1.0,0.00,5.966667
3,2,2020-12-31 23:57:51,2021-01-01 00:04:56,N,1.0,168,75,1.0,1.99,8.00,...,0.5,0.00,0.00,NaN,0.3,9.30,2.0,1.0,0.00,7.083333
4,2,2021-01-01 00:16:36,2021-01-01 00:16:40,N,2.0,265,265,3.0,0.00,-52.00,...,-0.5,0.00,0.00,NaN,-0.3,-52.80,3.0,1.0,0.00,0.066667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76513,2,2021-01-31 21:38:00,2021-01-31 22:16:00,NaN,NaN,81,90,NaN,17.63,56.23,...,0.0,0.00,6.12,NaN,0.3,65.40,NaN,NaN,NaN,38.000000
76514,2,2021-01-31 22:43:00,2021-01-31 23:21:00,NaN,NaN,35,213,NaN,18.36,46.66,...,0.0,12.20,6.12,NaN,0.3,65.28,NaN,NaN,NaN,38.000000
76515,2,2021-01-31 22:16:00,2021-01-31 22:27:00,NaN,NaN,74,69,NaN,2.50,18.95,...,0.0,0.00,0.00,NaN,0.3,22.00,NaN,NaN,NaN,11.000000
76516,2,2021-01-31 23:10:00,2021-01-31 23:37:00,NaN,NaN,168,215,NaN,14.48,48.87,...,0.0,0.00,6.12,NaN,0.3,58.04,NaN,NaN,NaN,27.000000


In [44]:
# df[df.trip_type == 2]

In [46]:
df.duration.describe(percentiles=[0.95, 0.98, 0.99])

count    76518.000000
mean        19.927896
std         59.338594
min          0.000000
50%         13.883333
95%         44.000000
98%         56.000000
99%         67.158167
max       1439.600000
Name: duration, dtype: float64

In [49]:
df = df[(df.duration >= 0) & (df.duration <= 60)] 

In [51]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

In [56]:
df.loc[:, categorical] = df[categorical].astype(str)

In [60]:
from sklearn.feature_extraction import DictVectorizer

In [64]:
train_dicts = df[categorical + numerical].to_dict(orient='records')

In [65]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [66]:
X_train

<75356x508 sparse matrix of type '<class 'numpy.float64'>'
	with 226068 stored elements in Compressed Sparse Row format>

In [67]:
dv.feature_names_

['DOLocationID=1',
 'DOLocationID=10',
 'DOLocationID=100',
 'DOLocationID=101',
 'DOLocationID=102',
 'DOLocationID=106',
 'DOLocationID=107',
 'DOLocationID=108',
 'DOLocationID=109',
 'DOLocationID=11',
 'DOLocationID=111',
 'DOLocationID=112',
 'DOLocationID=113',
 'DOLocationID=114',
 'DOLocationID=115',
 'DOLocationID=116',
 'DOLocationID=117',
 'DOLocationID=118',
 'DOLocationID=119',
 'DOLocationID=12',
 'DOLocationID=120',
 'DOLocationID=121',
 'DOLocationID=122',
 'DOLocationID=123',
 'DOLocationID=124',
 'DOLocationID=125',
 'DOLocationID=126',
 'DOLocationID=127',
 'DOLocationID=128',
 'DOLocationID=129',
 'DOLocationID=13',
 'DOLocationID=130',
 'DOLocationID=131',
 'DOLocationID=132',
 'DOLocationID=133',
 'DOLocationID=134',
 'DOLocationID=135',
 'DOLocationID=136',
 'DOLocationID=137',
 'DOLocationID=138',
 'DOLocationID=139',
 'DOLocationID=14',
 'DOLocationID=140',
 'DOLocationID=141',
 'DOLocationID=142',
 'DOLocationID=143',
 'DOLocationID=144',
 'DOLocationID=145',

In [ ]:
target = 'duration'
y_train = df[target].values

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

In [ ]:
y_pred =  lr.predict(X_train)

In [ ]:
sns.distplot(y_pred, label='prediction')
sns.distplot(y_train, label='actual')

plt.legend()

In [1]:
mean_squared_error(y_train, y_pred, squared=False)

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred =  lr.predict(X_train)